# import libs

In [1]:
import numpy as np
import pandas as pd
import torch
from train import TwoInputFCNN as FCNN


# import data

In [2]:
labels = np.genfromtxt('/home/ysy/ERIE/single_psm_manipulation_dataset/output/C_M1_T1_4/labels_30hz.txt', delimiter=',')
coordinates_L = pd.read_hdf('/home/ysy/ERIE/resources/C_M1_T1_4_L_h264DLC_resnet50_TrackJun13shuffle1_50000.h5')
coordinates_R = pd.read_hdf('/home/ysy/ERIE/resources/C_M1_T1_4_R_h264DLC_resnet50_TrackJun13shuffle1_50000.h5')


In [3]:
# unify size
frames = min(len(labels), len(coordinates_L), len(coordinates_R))


In [4]:
X_train_L = coordinates_L.filter(regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
X_train_R = coordinates_R.filter(regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]


In [5]:
y_train = labels[:frames, 6:9]


In [8]:
y_train

array([[ 5.22668997e-05,  1.46099253e-01, -5.50815757e-02],
       [ 1.48739346e-04,  1.46198152e-01, -5.50694759e-02],
       [-4.33166133e-06,  1.46261761e-01, -5.50527258e-02],
       ...,
       [-3.45088984e-03,  1.45966597e-01, -6.14891618e-02],
       [-3.70740076e-03,  1.45931765e-01, -6.19525046e-02],
       [-3.85850854e-03,  1.45585719e-01, -6.27472804e-02]])

In [6]:
X_train_L.shape, X_train_R.shape, y_train.shape

((3460, 16), (3460, 16), (3460, 3))

# Trainning

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = FCNN(input_dim=16, hidden_dim=64, output_dim=1, device=device)
net.train(X_train_R, X_train_L, y_train, epochs=1000)


/home/ysy/miniconda3/envs/torch-env/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32, 3])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (32) must match the size of tensor b (3) at non-singleton dimension 1

# Testing

In [ ]:
labels = np.genfromtxt('/home/ysy/ERIE/single_psm_manipulation_dataset/output/C_M1_T1_7/labels_30hz.txt', delimiter=',')
coordinates_L = pd.read_hdf('/home/ysy/ERIE/resources/C_M1_T1_7_L_h264DLC_resnet50_TrackJun13shuffle1_50000.h5')
coordinates_R = pd.read_hdf('/home/ysy/ERIE/resources/C_M1_T1_7_R_h264DLC_resnet50_TrackJun13shuffle1_50000.h5')


In [ ]:
# unify size
frames = min(len(labels), len(coordinates_L), len(coordinates_R))


In [ ]:
X_test_L = coordinates_L.filter(regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
X_test_R = coordinates_R.filter(regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]


In [ ]:
y_test = labels[:frames, 6:9]

In [ ]:
X_test_L.shape, X_test_R.shape, y_test.shape

((3540, 16), (3540, 16), (3540,))

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = net.predict(X_test_R, X_test_L)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on test data: {mse}')

Mean Squared Error on test data: 0.002306144461151742
